### Importações (Mongo)

In [2]:
import pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv
import os

### Criando conexão e dataframes iniciais com o MongoDB

In [3]:
# conexão com o MongoDB

connection_string = os.getenv('mongo_string')
client = MongoClient(connection_string)
db = client['teste_dados']

# Usado pra consultar os collections
# collections = db.list_collection_names()
# for colecao in collections:
#     print(colecao)

multas = db['multas']

multas_dados = list(multas.find())

df_multas = pd.DataFrame(multas_dados)
display(df_multas)


,_id,escopo_autuacao,mes,ano,uf,amparo_legal,descricao_infracao,quantidade_autos
0,6407641b305a576376e6f33d,CMT - Capacidade Máxima de Tração,JANEIRO,2021,MG,Lei 9503/97 Lei 10561/02 Res. CONTRAN nº 210/0...,Transitar com o veículo excedendo a CMT acima ...,2
1,6407641b305a576376e6f340,CMT - Capacidade Máxima de Tração,JANEIRO,2021,RJ,Lei 9503/97 Lei 10561/02 Res. CONTRAN nº 210/0...,Transitar com o veículo excedendo a CMT acima ...,3
2,6407641b305a576376e6f33e,CMT - Capacidade Máxima de Tração,JANEIRO,2021,MS,Lei 9503/97 Lei 10561/02 Res. CONTRAN nº 210/0...,Transitar com o veículo excedendo a CMT acima ...,1
3,6407641b305a576376e6f341,CMT - Capacidade Máxima de Tração,JANEIRO,2021,RJ,Lei 9503/97 Lei 10561/02 Res. CONTRAN nº 210/0...,Transitar com o veículo excedendo a CMT em até...,5
4,6407641b305a576376e6f344,Evasão de Balança,JANEIRO,2021,RJ,"Lei 9503 DE 23/09/1997 Transpor, sem autorizaç...",Art. 209 Deixar de adentrar as áreas destinada...,6330
...,...,...,...,...,...,...,...,...
965,6407641b305a576376e6f6f9,Excesso de Peso,DEZEMBRO,2022,PR,Lei 9503/97 Lei 10233/01 Res. CONTRAN nº 882/2...,Transitar com o veículo com excesso de peso - ...,360
966,6407641b305a576376e6f6ff,Excesso de Peso,DEZEMBRO,2022,RS,Lei 9503/97 Lei 10233/01 Res. CONTRAN nº 882/2...,Transitar com o veículo com excesso de peso - ...,1044
967,6407641b305a576376e6f700,Excesso de Peso,DEZEMBRO,2022,SC,Lei 9503/97 Lei 10233/01 Res. CONTRAN nº 882/2...,Transitar com o veículo com excesso de peso - ...,86
968,6407641b305a576376e6f701,Excesso de Peso,DEZEMBRO,2022,SC,Lei 9503/97 Lei 10233/01 Res. CONTRAN nº 882/2...,Transitar com o veículo com excesso de peso - ...,174


#### Acumulado de multas

In [4]:
###### Acumulado dia a dia e padronização

# padronizaçao do mês
meses_map = {
    'JANEIRO': 1,
    'FEVEREIRO': 2,
    'MARÇO': 3,
    'ABRIL': 4,
    'MAIO': 5,
    'JUNHO': 6,
    'JULHO': 7,
    'AGOSTO': 8,
    'SETEMBRO': 9,
    'OUTUBRO': 10,
    'NOVEMBRO': 11,
    'DEZEMBRO': 12
}

df_multas['month_num'] = df_multas['mes'].map(meses_map)

def validMonth(month): # padroniza o valor de year_month com um 0 para caso o valor do mes seja menor q 10
    return month.zfill(2)

df_multas['year_month'] = df_multas['ano'].astype(str) + df_multas['month_num'].astype(str).apply(validMonth)

# ordena pela coluna período
df_multas = df_multas.sort_values('year_month', ascending=True)

# gravação após tratamento do mes/ano
df_multas.to_csv('tickets_data.csv',index=False)

# tratamento e construção de colunas calculadas
df_multas['cumulative_tickets'] = df_multas['quantidade_autos'].astype(int).cumsum() # Acumulado de multas
df_multas['previous_tickets'] = df_multas['cumulative_tickets'].shift().fillna(0).astype(int) # acumulado no periodo anterior
df_multas['diff_by_period'] = df_multas['cumulative_tickets'] - df_multas['previous_tickets']

df_tickets_cumulative = df_multas[['year_month','quantidade_autos','cumulative_tickets','previous_tickets','diff_by_period']]

df_tickets_cumulative.to_csv('tickets_cumulative.csv',index=False)
display(df_tickets_cumulative)

,year_month,quantidade_autos,cumulative_tickets,previous_tickets,diff_by_period
0,202101,2,2,0,2
68,202101,57,59,2,57
69,202101,58,117,59,58
70,202101,17,134,117,17
71,202101,154,288,134,154
...,...,...,...,...,...
783,202212,5,376185,376180,5
784,202212,2,376187,376185,2
785,202212,61,376248,376187,61
620,202212,249,376497,376248,249


In [5]:
##### Agrupando e exportando por 

df_tickets_grouped = df_tickets_cumulative.groupby('year_month')['cumulative_tickets'].max().reset_index() # agrupa e retorna valor máximo no período
df_tickets_grouped['previous_period'] = df_tickets_grouped['cumulative_tickets'].shift(1) # traz o valor do período anterior em uma nova coluna
df_tickets_grouped['diff_by_period'] = df_tickets_grouped['cumulative_tickets'] - df_tickets_grouped['previous_period'] # diferença entre períodos
df_tickets_grouped['year'] = df_tickets_grouped['year_month'].astype(str).str.slice(0,4)
df_tickets_grouped['week'] = df_tickets_grouped['year_month'].astype(str).str.slice(-2)

df_tickets_grouped.to_csv('tickets_cumulative_grouped.csv', index=False) # export csv

pd.set_option('display.max_rows', 6) 
display(df_tickets_grouped)

,year_month,cumulative_tickets,previous_period,diff_by_period,year,week
0,202101,14649,NaN,NaN,2021,01
1,202102,30018,14649.0,15369.0,2021,02
2,202103,47605,30018.0,17587.0,2021,03
...,...,...,...,...,...,...
21,202210,350176,336580.0,13596.0,2022,10
22,202211,366289,350176.0,16113.0,2022,11
23,202212,376515,366289.0,10226.0,2022,12
